In [8]:
import os
import numpy as np
import h5py as h
import fnmatch as fm

In [9]:
filename = "nreo.hdf5"#"../2022-01-01_Scan/nreo_NR_both_diver_c15_only_Halo_WW/samples/nreo.hdf5"
rawH5 = h.File(filename, "r")
namesplit = filename.split("_")
coupling = 4#int(namesplit[namesplit.index("diver")+1].strip("c"))
# rawH5["nreo"].keys()

In [10]:
totalloglike = "LogLike"
# loglikes = fm.filter((fm.filter(rawH5["nreo"].keys(), "*_Log*")), "[!Runtime]*_isvalid")
# loglikes = [s.replace("_isvalid", "") for s in loglikes]#.append("#IC79_loglike @DarkBit::IC79_loglike")
# loglikes.append("#IC79_loglike @DarkBit::IC79_loglike")
# print(len(loglikes), loglikes)
# priorlikes = fm.filter((fm.filter(rawH5["nreo"].keys(), "*lnL_*")), "[!Runtime]*[!id]")
# print(len(priorlikes), priorlikes)
subloglike = ['#CDMSlite_LogLikelihood @DarkBit::CDMSlite_GetLogLikelihood', '#CRESST_II_LogLikelihood @DarkBit::CRESST_II_GetLogLikelihood', '#DarkSide_50_LogLikelihood @DarkBit::DarkSide_50_GetLogLikelihood', '#LUX_2016_LogLikelihood @DarkBit::LUX_2016_GetLogLikelihood', '#PICO_60_2017_LogLikelihood @DarkBit::PICO_60_2017_GetLogLikelihood', '#PandaX_2016_LogLikelihood @DarkBit::PandaX_2016_GetLogLikelihood', '#PandaX_2017_LogLikelihood @DarkBit::PandaX_2017_GetLogLikelihood', '#XENON1T_2018_LogLikelihood @DarkBit::XENON1T_2018_GetLogLikelihood', '#IC79_loglike @DarkBit::IC79_loglike', '#lnL_rho0 @DarkBit::lnL_rho0_lognormal', '#lnL_v0 @DarkBit::lnL_v0_gaussian', '#lnL_vesc @DarkBit::lnL_vesc_gaussian']

In [11]:
# for i in range(len(rawH5["nreo/"+totalloglike])):
#     total = rawH5["nreo/"+totalloglike][i]
#     summation = 0
#     print("Total: "+str(total), end="\t")
#     for sublike in subloglike:
#         summation = summation + rawH5["nreo/"+sublike][i]
#     print("Summation: "+str(summation), end="\t")
#     print("Diff: "+str(total-summation))

In [12]:
filename = "nreo_NR_both_diver_c4_only_Halo_WW.dat"
H5file = "nreo_NR_both_diver_c4_only_Halo_WW.hdf5"
print("Step 1")
pointID, logL, v0, vesc, rho0, dmMass, c0_i, captures, maxcaps, newANTARES, newIceCube = np.loadtxt(filename, dtype=float).T
print("Step 2")
appendingH5 = h.File(H5file, "a")
print("Step 3")
points = appendingH5["nreo/pointID"]
length = len(points)
completeANTARES = np.zeros(length, dtype=float)
completeIceCube = np.zeros(length, dtype=float)
modifiedLike = np.zeros(length, dtype=float)
ANTARES_isvalid = np.zeros(length, dtype=int)
IceCube_isvalid = np.zeros(length, dtype=int)
modifiedLike_isvalid = np.zeros(length, dtype=int)
recalcCaps = np.zeros(length, dtype=float)
for i in range(len(points)):
    point = i+1
    if point in pointID:
        indexPoint = np.where(pointID == point)[0][0]
        completeANTARES[i] = newANTARES[indexPoint]
        ANTARES_isvalid[i] = 1
        completeIceCube[i] = newIceCube[indexPoint]
        IceCube_isvalid[i] = 1
        modifiedLike[i] = logL[indexPoint] + newANTARES[indexPoint] + newIceCube[indexPoint]
        modifiedLike_isvalid[i] = 1
        recalcCaps[i] = captures[indexPoint]

Step 1
Step 2
Step 3


In [61]:
appendingH5 = h.File(H5file, "a")

In [62]:
del appendingH5["nreo/ANTARES_new_LogLike"]
del appendingH5["nreo/ANTARES_new_LogLike_isvalid"]
del appendingH5["nreo/IceCube_new_LogLike"]
del appendingH5["nreo/IceCube_new_LogLike_isvalid"]
del appendingH5["nreo/Modified_LogLike"]
del appendingH5["nreo/Modified_LogLike_isvalid"]
del appendingH5["nreo/Recalculated_Capturerate"]

In [63]:
appendingH5.create_dataset(name="nreo/ANTARES_new_LogLike", data=completeANTARES)
appendingH5.create_dataset(name="nreo/ANTARES_new_LogLike_isvalid", data=ANTARES_isvalid)
appendingH5.create_dataset(name="nreo/IceCube_new_LogLike", data=completeIceCube)
appendingH5.create_dataset(name="nreo/IceCube_new_LogLike_isvalid", data=IceCube_isvalid)
appendingH5.create_dataset(name="nreo/Modified_LogLike", data=modifiedLike)
appendingH5.create_dataset(name="nreo/Modified_LogLike_isvalid", data=modifiedLike_isvalid)
appendingH5.create_dataset(name="nreo/Recalculated_Capturerate", data=recalcCaps)

<HDF5 dataset "Recalculated_Capturerate": shape (200000,), type "<f8">

In [64]:
print(appendingH5["nreo/Recalculated_Capturerate"][1])

2.891e+21


In [65]:
appendingH5.close()